#### Question 1 (10 Points)
**List as many use cases for the dataset as possible.**
1. Group cars with similar features to display together on the website.
2. `Price` Prediction
3. `Normalized-losses` prediction. We can see from the sample data thare are many NaN values for this fields. 
4. `symboling` classification. 
4. Create embedings to each car to be used in other models that use the car as a feature.
5. Denoizer to fill the NaN values to improve data quality. 

#### Question 2 (10 Points)
**Auto1 has a similar dataset (yet much larger...) 
Pick one of the use cases you listed in question 1 and describe how building a statistical model based on the dataset could best be used to improve Auto1’s business.**

I chose the price prediction. I believe this is one of the most import prediction Auto1 has.  Since Auto1 has a similar dataset but much bigger the one for this test, I would suggest using an ensemble of different models, from linear models like linear regressions to non-linear, tree-based methods and  Neural networks. With this approach, we can extract the maximum of information from the limited data.

#### Question 3 (20 Points)
**Implement the model you described in question 2 in R or Python. The code has to retrieve the data, train and test a statistical model, and report relevant performance criteria. **

Check below.

#### Question 4 (60 Points)
**A. Explain each and every of your design choices (e.g., preprocessing, model selection, hyper parameters, evaluation criteria). Compare and contrast your choices with alternative methodologies.** 
Here I will give a summary of the idea. For more insights, please refer to the code below. 

1. preprocessing: Since we have limited data, for both, number of samples and num. of features, I did a small data preparation:
   1. removed Nan-values from the target `price`, created categorical variables and a binary encoding for those. 
   1. renamed the columns with a slash.
   1. shuffled the data frame to avoid any bias in the model.
    
1. model selection: Stacking (also called meta ensembling) is a model ensembling technique used to combine information from multiple predictive models to generate a new model. Stacking is most effective when the base models are significantly different. To create different models I used two different methods, lightgbm (gradient boosting framework) and Ordinary Least Squares (OLS).  For each method (lightgbm and OLS) I run 15 times changing the features used.   Due to the lack of samples, I chose to use a k-fold validation with `k=5` and a holdout of 10% of the samples for testing. I noticed a lot of variance between each fold (due to the lack of sample). I should use more folds to overcome this. Although, I am using ensembles, so I expect that this variance will be minimized.

1. hyperparameters: For lightgbm, I tried to use small trees and col samples to increase the model diversity. I have not performed any parameter tuning, Usually, I would use `skopt` (https://scikit-optimize.github.io/), but due to time limitation, I use some standard values. I have run the lightgbm model with early stopping to get some intuition of the number of boosting iterations before the final version. But, again, the variance between folds was very large, so I decided to keep the early stopping for the final model. 

1. evaluation criteria: I used root mean square error for evaluation and loss function.


**B. Describe how you would improve the model in Question 3 if you had more time.**

1. since we have very few samples, I would change the 5-fold to a leave-one-out cross-validation. 
2. find optimal parameters for the lightgbm models
3. try to fill NaN values in the dataset.
4. More types of encodings for the categorical variables to increase the diversity of models. 
5. use xgboost
6. use lasso and ridge regression, then I need to normalize the data due to their regularization. 
7. change the final staking model since it was not better than the average of the models. 
8. feature engineering for the dataset and the final stack




In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
import lightgbm as lgb
import category_encoders as ce
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression

% matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
filename = 'data.csv'
dtypes = {'symboling': int,'normalized-losses': np.float, 'make': str,
          'fuel-type': str, 'aspiration': str, 'num-of-doors': str,
          'body-style': str, 'drive-wheels': str, 'engine-location': str, 
          'wheel-base': np.float32, 'length': np.float32,'width': np.float32,
          'height': np.float32, 'curb-weight': np.float32, 'engine-type': str,
          'num-of-cylinders': str, 'engine-size': np.float32, 
          'fuel-system': str, 'bore': np.float32, 'stroke': np.float32,
          'compression-ratio': np.float32,  'horsepower': np.float32, 
          'peak-rpm': np.float32, 'city-mpg': np.float32,
          'highway-mpg': np.float32, 'price': np.float32,
         }

df = pd.read_csv(filename, na_values='?',dtype=dtypes)
df =  df.sample(frac=1, random_state=42)
df.dropna(axis=0, subset=['price'], inplace=True)  #drop NaN since it is the target value.
df.reset_index(drop=True, inplace=True)
y = df['price']
df.drop(columns=['price'], inplace=True)
df.columns = [i.replace("-", "_")for i in df.columns]  #change the columns names to be able to use with OLS.

categorical_feature = [col for col in df.columns 
                       if df.dtypes[col] == object]

not_categorical = [col for col in df.columns 
                   if col not in categorical_feature]

Here we will transform the categorical variables to type category in pandas and also create a binary encoding for each categorical variable.
OBS: we are going to use lightgbm, it can handle categorical variables, but I also chose to have binary encoded variables.
We will use it to add some variety of features. 

In [3]:
bin_encoded_cols = []
for col in categorical_feature:
    uniques = pd.DataFrame(list(df[col].unique()), columns = [col])
    enc = ce.BinaryEncoder(verbose=1, cols=[col])
    uniques = pd.concat([uniques,enc.fit_transform(uniques)], axis=1)
    df= df.merge(uniques, how = 'left', on = col )
    bin_encoded_cols.extend(list(uniques.columns[1:]))
    df[col] = df[col].astype('category')
    
    

In [4]:
#10% of the data to test. 
X_train, X_test, y_train, y_test = train_test_split( df, y, test_size=0.10, random_state=42)
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

In [5]:
#main function to train the lightgbm models
def train_model_LGBM(data_, y_ ,test_, folds_,feats = None, seed= 42, params_lgbm=None ):
    if not params_lgbm:
        params_lgbm = {
                        'learning_rate': 0.01,
                        'task': 'train',
                        'boosting_type': 'gbdt',
                        'objective': 'regression_l2',
                        'metric': {'rmse'},
                        'num_leaves': 50,
                        'max_depth': 5,
                        'feature_fraction' :.8,
                        'seed':seed
                        }

    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    
    if not feats:
        feats = list(data_.columns)
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_, y_)):
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]
        
        d_train = lgb.Dataset(trn_x, trn_y.values)
        d_test = lgb.Dataset(val_x, val_y.values, reference = d_train)

        bst = lgb.train(params_lgbm, d_train, 2000, valid_sets = [d_train,d_test], valid_names =['train','test'],
                early_stopping_rounds=50, verbose_eval=1000)
        
        

        oof_preds[val_idx] = bst.predict(val_x, num_iteration=bst.best_iteration)
        sub_preds += bst.predict(test_[feats],
                                 num_iteration=bst.best_iteration) / folds_.n_splits

        print('Fold %2d MSE : %.6f' %
              (n_fold + 1, np.sqrt(mean_squared_error(val_y, oof_preds[val_idx]))))


    print('Full MSE score %.6f' % np.sqrt(mean_squared_error(y_, oof_preds)))

    return oof_preds, sub_preds

In [6]:
#the folds must be the same for all the models (OLS and lightgbm)
kf = KFold(n_splits=5)
num_models = 15

In [7]:
oof_preds_concat = []
sub_preds_concat = []
np.random.seed(42) #reproductibility of `np.random.choice`
for i in range(num_models):
    #each iteraction we use a different seed for LGBM
    print(f'model {i+1}')
    feats = not_categorical+ np.random.choice([categorical_feature, bin_encoded_cols] ) #choose which type of categorical to use.
    oof_preds,sub_preds = train_model_LGBM(X_train, y_train, X_test, kf,feats=feats, seed=i)
    oof_preds_concat.append(oof_preds)
    sub_preds_concat.append(sub_preds)
    
oof_preds_concat = np.column_stack(oof_preds_concat)
sub_preds_concat = np.column_stack(sub_preds_concat)

model 1
Training until validation scores don't improve for 50 rounds.
[1000]	train's rmse: 1243.83	test's rmse: 2362.63
Early stopping, best iteration is:
[1804]	train's rmse: 964.934	test's rmse: 2281.4
Fold  1 MSE : 2239.020406
Training until validation scores don't improve for 50 rounds.
[1000]	train's rmse: 1175.98	test's rmse: 2728.8
[2000]	train's rmse: 852.471	test's rmse: 2647.4
Fold  2 MSE : 2309.979774
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[912]	train's rmse: 1472	test's rmse: 1891.93
Fold  3 MSE : 1922.413091
Training until validation scores don't improve for 50 rounds.
[1000]	train's rmse: 1338.63	test's rmse: 2584.88
Early stopping, best iteration is:
[1391]	train's rmse: 1155.83	test's rmse: 2502.17
Fold  4 MSE : 2713.261496
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[286]	train's rmse: 2424.37	test's rmse: 2166.66
Fold  5 MSE : 2166.333497
Full MSE score 2284.

Fold  4 MSE : 2693.956982
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[285]	train's rmse: 2419.25	test's rmse: 2169.25
Fold  5 MSE : 2169.097999
Full MSE score 2293.533474
model 9
Training until validation scores don't improve for 50 rounds.
[1000]	train's rmse: 1250.47	test's rmse: 2343.38
Early stopping, best iteration is:
[1822]	train's rmse: 954.388	test's rmse: 2259.92
Fold  1 MSE : 2224.420324
Training until validation scores don't improve for 50 rounds.
[1000]	train's rmse: 1180.8	test's rmse: 2739.18
Early stopping, best iteration is:
[1434]	train's rmse: 1003.61	test's rmse: 2697.1
Fold  2 MSE : 2411.424318
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[925]	train's rmse: 1462.96	test's rmse: 1892.89
Fold  3 MSE : 1908.011759
Training until validation scores don't improve for 50 rounds.
[1000]	train's rmse: 1336.74	test's rmse: 2579.08
Early stopping, best iteration is:
[152

In [8]:
#here some insights from the first stacking.
print('Simple ensemble using the average of all Tree-based models = {}'.format(np.sqrt(mean_squared_error(y_test, sub_preds_concat.mean(axis=1))) ))
lr = LinearRegression()
lr.fit(oof_preds_concat , y_train)
final_test_preds = lr.predict(sub_preds_concat) 
print('Using a linear regression = {}'.format(np.sqrt(mean_squared_error(y_test, final_test_preds)) ))


Simple ensemble using the average of all Tree-based models = 1908.0244179521724
Using a linear regression = 2281.5258448646937


In [9]:
#main function to train the OLS models
def train_model_OLS(data_, y_ ,test_, folds_,formula ):
    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_, y_)):
        trn_x, trn_y = data_.iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_.iloc[val_idx], y_.iloc[val_idx]

        linear_model = smf.ols(formula=formula, data=pd.concat([trn_x,trn_y], axis=1))
        linear_model_fit = linear_model.fit()

        oof_preds[val_idx] = linear_model_fit.predict(val_x)
        sub_preds += linear_model_fit.predict(test_) / folds_.n_splits

        print('Fold %2d MSE : %.6f' %
              (n_fold + 1, np.sqrt(mean_squared_error(val_y, oof_preds[val_idx]))))


    print('Full MSE score %.6f' % np.sqrt(mean_squared_error(y_, oof_preds)))

    return oof_preds, sub_preds

In [10]:
#here we removed the columns with NaN to avoid numerical problems. 
categorical_options = ['make', 'fuel_type', 'aspiration', 'body_style', 'drive_wheels', 'engine_location', 'engine_type',
                       'num_of_cylinders', 'fuel_system']

numerical_options = ['symboling','wheel_base', 'length','width','height','curb_weight','engine_size','compression_ratio',
                     'city_mpg','highway_mpg']

oof_preds_concat_ols = []
sub_preds_concat_ols = []
num_models = 15
np.random.seed(42) #reproductibility of `np.random.choice`
for i in range(num_models):
    print(f'model {i+1}')
    #for each model we will use a set of features randomly chosen
    feat1, feat2, feat3, feat4 = np.random.choice(numerical_options, 4, replace=False )
    feat5, feat6, feat7 = np.random.choice(categorical_options, 3, replace=False )
    formula =f"price ~  C({feat5}) + C({feat6}) + C({feat7}) + {feat1} + {feat2} + {feat3} + {feat4}"
    oof_preds,sub_preds = train_model_OLS(X_train, y_train, X_test, kf,formula=formula)
    oof_preds_concat_ols.append(oof_preds)
    sub_preds_concat_ols.append(sub_preds)
    
oof_preds_concat_ols = np.column_stack(oof_preds_concat_ols)
sub_preds_concat_ols = np.column_stack(sub_preds_concat_ols)

model 1
Fold  1 MSE : 3566.139419
Fold  2 MSE : 3145.413055
Fold  3 MSE : 2261.861477
Fold  4 MSE : 2615.892269
Fold  5 MSE : 2118.396175
Full MSE score 2794.905569
model 2
Fold  1 MSE : 3991.102978
Fold  2 MSE : 3047.178470
Fold  3 MSE : 2918.456641
Fold  4 MSE : 3374.730076
Fold  5 MSE : 2536.560743
Full MSE score 3211.060020
model 3
Fold  1 MSE : 3805.712026
Fold  2 MSE : 2892.688511
Fold  3 MSE : 3638.810477
Fold  4 MSE : 3285.454869
Fold  5 MSE : 2960.610097
Full MSE score 3336.209178
model 4
Fold  1 MSE : 3641.179782
Fold  2 MSE : 3004.265768
Fold  3 MSE : 2986.794304
Fold  4 MSE : 3233.472781
Fold  5 MSE : 3131.904579
Full MSE score 3208.392611
model 5
Fold  1 MSE : 3226.512915
Fold  2 MSE : 2689.909422
Fold  3 MSE : 2862.500140
Fold  4 MSE : 2958.967418
Fold  5 MSE : 2071.054202
Full MSE score 2788.715081
model 6
Fold  1 MSE : 4592.834610
Fold  2 MSE : 3878.281111
Fold  3 MSE : 4755.526144
Fold  4 MSE : 3876.808581
Fold  5 MSE : 4297.659622
Full MSE score 4295.340406
model 7
Fo

In [11]:
print('Simple ensemble using the average of all OLS models = {}'.format(np.sqrt(mean_squared_error(y_test, sub_preds_concat_ols.mean(axis=1))) ))
lr = LinearRegression()
lr.fit(oof_preds_concat_ols , y_train)
final_test_preds = lr.predict(sub_preds_concat_ols) 
print('Using a linear regression for each model = {}'.format(np.sqrt(mean_squared_error(y_test, final_test_preds)) ))


Simple ensemble using the average of all OLS models = 2456.8550811469754
Using a linear regression for each model = 2265.701447161632


Puting everthing together..

In [12]:
oof_preds =  np.concatenate([oof_preds_concat, oof_preds_concat_ols], axis=1)
sub_preds =  np.concatenate([sub_preds_concat, sub_preds_concat_ols], axis=1)

In [13]:
print('Simple ensemble using the average of OLS+tree models = {}'.format(np.sqrt(mean_squared_error(y_test, sub_preds.mean(axis=1))) ))

lr = LinearRegression()
lr.fit(oof_preds , y_train)
final_test_preds = lr.predict(sub_preds) 
print('Using a linear regression = {}'.format(np.sqrt(mean_squared_error(y_test, final_test_preds)) ))


Simple ensemble using the average of OLS+tree models = 2078.0432675879183
Using a linear regression = 2611.756699187603


### Final conclusion


We noticed a lot of variance between the folds, we could use more folds. 
The final toot mean square error was good, we have a mean price value of `$13207.12` we got RMSE of `$2078.04`

we could improve the stacking using other than LinearRegression or mean value, e.g., xgboost, lightgbm, NN, SVR etc. Also, since we noticed that the performance of the lightgbm is better than the OLS, we could weight its models higher than the OLS ones. 